<a href="https://colab.research.google.com/github/ksushenc1a/NLP-course/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install aiogram==2.25.2
import json
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.utils import executor
from aiogram.contrib.fsm_storage.memory import MemoryStorage

# Установите ваш API токен здесь
API_TOKEN = ""

# Настройка логирования
logging.basicConfig(level=logging.INFO)

# Инициализация бота и диспетчера
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

# Загрузка FAQ из JSON
with open('faq.json', 'r', encoding='utf-8') as f:
    data = json.load(f)['faq']  # Изменено для получения списка FAQ

# Кнопки
button_company = types.KeyboardButton("О компании")
button_contact = types.KeyboardButton("Связаться с оператором")
keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True).add(button_company, button_contact)

# Обработка команд /start
@dp.message_handler(commands=['start'])
async def send_welcome(message: types.Message):
    await message.reply("Добро пожаловать! Какой у вас вопрос?", reply_markup=keyboard)

# Обработка текстовых сообщений
@dp.message_handler()
async def handle_message(message: types.Message):
    user_message = message.text.lower()

    # Проверка на кнопки
    if user_message == "о компании":
        await message.reply("Наша компания занимается доставкой товаров по всей стране.")
        return
    elif user_message == "связаться с оператором":
        await message.reply("Перевожу на оператора…")
        return

    # Ключевые слова для поиска вопросов
    keywords = {
        "Цены": ["цены", "стоимость", "заказ", "оплата"],
        "Часы работы": ["часы работы", "время работы", "доступность"],
        "Доставка": ["доставка", "сроки доставки", "стоимость доставки", "отслеживание"],
        "Возврат": ["возврат", "обмен", "возврат товара", "гарантия"],
        "Контакты": ["связаться", "телефон", "email", "адрес"]
    }
# Поиск ответа по ключевым словам
    for word in user_message.split():
        for category, keys in keywords.items():
            if word in keys:
                # Также проверяем на соответствие с вопросами в FAQ
                for item in data:
                    if category.lower() in item['question'].lower():
                        await message.reply(item['answer'])
                        return

    # Перебор вопросов из faq.json для более глубокого поиска
    for entry in data:
        if entry['question'].lower() in user_message:
            await message.reply(entry['answer'])
            return

    # Если ничего не подошло
    await message.reply("Извините, я не понимаю ваш запрос.")

async def main():
    bot = Bot(token=API_TOKEN)
    await dp.start_polling(bot)

# Проверяем, запущен ли скрипт напрямую (не импортирован в другой файл)
if __name__ == "__main__":
    # Запускаем основную функцию
    await main()

CancelledError: 